In [2]:
# UNCOMMENT THESE TO INSTALL THE REQUIRED LIBRARIES

# %pip install pandas
# %pip install tensorflow
# %pip install scikit-learn
# %pip install pymysql
# %pip install sshtunnel

In [3]:
import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf
import pymysql
import sshtunnel

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import load_model

print("Numpy version:", np.__version__)
print("Pandas version:", pd.__version__)
print("Tensorflow version:", tf.__version__)
print("Sklearn version:", sklearn.__version__)
print("Pymysql version:", pymysql.__version__)
print("Sshtunnel version:", sshtunnel.__version__)

Numpy version: 1.26.4
Pandas version: 2.2.2
Tensorflow version: 2.16.1
Sklearn version: 1.4.2
Pymysql version: 1.4.6
Sshtunnel version: 0.4.0


### Reading data from a SQL-SERVER

In [4]:
# Set up SSH tunnel
server = sshtunnel.SSHTunnelForwarder(
    ('selene.hud.ac.uk', 22),
    ssh_username='workit',
    ssh_password='umbra(despair>Quartz218',
    remote_bind_address=('localhost', 3306)
)
server.start()

# Defining the connection
connection = pymysql.connect(host='localhost', port=server.local_bind_port, user='workit', password='umbra(despair>Quartz218', database='workit')

In [5]:
MODEL_PATH = "ann_gym_recommender.keras"

# Load the dataset
dataset = pd.read_sql_query("SELECT * FROM `megaGymDataset`", connection)
dataset.head()

,ID,Title,Desc,Type,BodyPart,Equipment,Level,Rating
0,0,Partner plank band row,The partner plank band row is an abdominal exe...,Strength,Abdominals,Bands,Intermediate,0.0
1,1,Banded crunch isometric hold,The banded crunch isometric hold is an exercis...,Strength,Abdominals,Bands,Intermediate,4.5
2,2,FYR Banded Plank Jack,The banded plank jack is a variation on the pl...,Strength,Abdominals,Bands,Intermediate,4.5
3,3,Banded crunch,The banded crunch is an exercise targeting the...,Strength,Abdominals,Bands,Intermediate,4.5
4,4,Crunch,The crunch is a popular core exercise targetin...,Strength,Abdominals,Bands,Intermediate,4.5


In [6]:
server.stop()

In [9]:
dataset.iloc[0][2]

'The partner plank band row is an abdominal exercise where two partners perform single-arm planks while pulling on the opposite ends of an exercise band. This technique can be done for time or reps in any ab-focused workout.'

In [6]:
def labelEncodersScaler(data):
    label_encoders = {}
    for feature in ["Type", "BodyPart", "Equipment", "Level"]:
        label_encoders[feature] = LabelEncoder()
        data[feature] = label_encoders[feature].fit_transform(data[feature])

    X = data[["Type", "BodyPart", "Equipment", "Level"]]
    y = data["ID"]
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Standardize the features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)

    return label_encoders, scaler

In [7]:
def recommend_exercises(user_input, model, data):
    label_encoders, scaler = labelEncodersScaler(data)
    
    user_input_encoded = [label_encoders[feature].transform([user_input[i]])[0] for i, feature in enumerate(["Type", "BodyPart", "Equipment", "Level"])]
    user_input_scaled = scaler.transform([user_input_encoded])
    
    predictions = model.predict(user_input_scaled)
    
    return data.iloc[np.squeeze(predictions.argsort())][::-1][:10].sort_values(["Rating"], ascending=False).index.to_list()

### Predict from user input

In [8]:
model = load_model(MODEL_PATH)
user_input = ["Strength", "Biceps", "Body Only", "Intermediate"]
IDs = recommend_exercises(user_input, model, dataset.copy())
dataset.iloc[IDs]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step


,ID,Title,Desc,Type,BodyPart,Equipment,Level,Rating
853,853,TBS Chin-Up,The a chin-up is a variation of the pull-up ex...,Strength,Biceps,Body Only,Intermediate,9.0
855,855,UN Pull-Up Supinated,Pull-up variation with a supinated (underhand)...,Strength,Biceps,Body Only,Intermediate,9.0
854,854,Pull-up - Gethin Variation,The pull-up is a multi-joint bodyweight exerci...,Strength,Biceps,Body Only,Intermediate,9.0
1044,1044,1.5-rep push-up,The 1.5-rep push-up is a more difficult variat...,Strength,Chest,Body Only,Intermediate,8.19
911,911,HM Left Calf-Spike Sumo Pulse,Dynamic calf exercise involving pulsing moveme...,Strength,Calves,Body Only,Intermediate,6.870833333333334
909,909,Standing Calf Raise,The standing calf raise is a popular movement ...,Strength,Calves,Body Only,Intermediate,6.870833333333333
910,910,Standing calf raise-,The standing calf raise is a popular movement ...,Strength,Calves,Body Only,Intermediate,6.870833333333333
694,694,HM Left Leg Swing,Dynamic leg swing exercise targeting hip flexo...,Strength,Adductors,Body Only,Intermediate,4.816666666666667
695,695,UNS Foam Roll Adductor,Using a foam roller to massage and release ten...,Strength,Adductors,Body Only,Intermediate,4.816666666666667
697,697,Side-lying leg adduction scissor,The side-lying leg adduction scissor is a dyna...,Strength,Adductors,Body Only,Intermediate,4.816666666666666


In [9]:
import tensorflowjs

ModuleNotFoundError: No module named 'tensorflowjs'

In [ ]:
%pip install tensorflowjs

In [ ]:
!tensorflowjs_converter --input_format keras "ann_gym_recommender.keras" ./jsmodel